<a href="https://colab.research.google.com/github/Kutloano-maekane/fcc/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep ='\t')
test_df = pd.read_csv(test_file_path, sep = '\t')
train_df = train_df.rename(columns = {'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.': 'message'})
test_df
test_df = test_df.rename(columns = {'i am in hospital da. . i will return home in evening': 'message'})
test_df

In [ ]:
test_df['label'] = test_df['ham'].map({'ham': 0, 'spam': 1})
test_labels = test_df['label']
test_df = test_df.drop(columns=['label'])
test_df


In [ ]:
X_train = train_df['message']
Y_train = labels
X_test = test_labels
Y_test = test_df['message']


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['message'])

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(train_df['message'])
X_test_seq = tokenizer.texts_to_sequences(test_df['message'])

# Pad
max_length = 100

X_train = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Embedding(10000, 32, input_length=100),
    layers.GlobalAveragePooling1D(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train,
    Y_train,
    epochs=10,
    validation_data=(X_test, test_labels),
    verbose=2
)


In [ ]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, test_labels))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(messages):
    # Check if it's a single message (string) or a list
    single_input = isinstance(messages, str)

    # Convert to list if it's a single string
    if single_input:
        messages = [messages]

    # Tokenize and pad
    sequences = tokenizer.texts_to_sequences(messages)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequences, maxlen=100, padding='post', truncating='post'
    )

    # Predict
    preds = model.predict(padded)

    # Convert predictions to labels
    results = []
    for pred in preds:
        p = float(pred[0])
        label = 'spam' if p > 0.5 else 'ham'
        results.append([p, label])

    # Return single result if input was a string
    return results[0] if single_input else results


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
